### **Personalized Recommendations with Pandas Corrwith method**

In [ ]:
import pandas as pd

In [ ]:
movies = pd.read_csv('movies.csv', usecols=['movieId', 'title'])
movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [ ]:
ratings = pd.read_csv('ratings.csv', usecols=['userId', 'movieId', 'rating'])
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
df = pd.merge(movies, ratings, on='movieId')
df.tail()

,movieId,title,userId,rating
100831,193581,Black Butler: Book of the Atlantic (2017),184,4.0
100832,193583,No Game No Life: Zero (2017),184,3.5
100833,193585,Flint (2017),184,3.5
100834,193587,Bungo Stray Dogs: Dead Apple (2018),184,3.5
100835,193609,Andrew Dice Clay: Dice Rules (1991),331,4.0


In [ ]:
user_rating = df.pivot_table(index='userId', columns='title', values='rating')

In [ ]:
selected_rating = user_rating["Can't Hardly Wait (1998)"]

In [ ]:
similar_movies = user_rating.corrwith(selected_rating)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [ ]:
corr = pd.DataFrame(similar_movies, columns=["Correlation"])

In [ ]:
corr = corr.dropna()

In [ ]:
corr.head(20)

,Correlation
title,
"'burbs, The (1989)",0.205439
(500) Days of Summer (2009),0.516398
*batteries not included (1987),0.666667
10 Things I Hate About You (1999),0.707107
"10,000 BC (2008)",-1.000000
101 Dalmatians (1996),0.828079
101 Dalmatians (One Hundred and One Dalmatians) (1961),0.925926
13 Going on 30 (2004),0.752525
"13th Warrior, The (1999)",0.654654


In [ ]:
corr = corr.sort_values('Correlation', ascending=False)

In [ ]:
corr.head(10)

,Correlation
title,
Lady Bird (2017),1.0
Regarding Henry (1991),1.0
Pumping Iron (1977),1.0
Down with Love (2003),1.0
Pumpkin (2002),1.0
Down and Out in Beverly Hills (1986),1.0
Punchline (1988),1.0
Quantum of Solace (2008),1.0
Quicksilver (1986),1.0


In [ ]:
corr.tail(10)

,Correlation
title,
Meet the Robinsons (2007),-1.0
RoboCop 2 (1990),-1.0
Bait (2000),-1.0
Chasing Liberty (2004),-1.0
Kinsey (2004),-1.0
Role Models (2008),-1.0
Warm Bodies (2013),-1.0
American Outlaws (2001),-1.0
Miami Vice (2006),-1.0


### **Personalized Recommendations with Tensorflow Recommender**

In [ ]:
!pip install tensorflow-recommenders

     |████████████████████████████████| 85 kB 2.9 MB/s 
     |████████████████████████████████| 462 kB 37.6 MB/s 


In [ ]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [ ]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/movielens/100k-ratings/0.1.0.incompleteCBFCOE/movielens-train.tfrecord


...

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/movielens/100k-movies/0.1.0.incompleteO737E6/movielens-train.tfrecord


  0%|          | 0/1682 [00:00<?, ? examples/s]

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.0. Subsequent calls will reuse this data.


In [ ]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

In [ ]:
#create our train/test split
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [ ]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

In [ ]:
embedding_dimension = 32

In [ ]:
# Build flexible representation models.

user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [ ]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [ ]:
from typing import Dict, Text
import pprint
import tempfile

In [ ]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

In [ ]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [ ]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [ ]:
model = MovielensModel(user_model, movie_model)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [ ]:
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 28s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0021 - factorized_top_k/top_5_categorical_accuracy: 0.0188 - factorized_top_k/top_10_categorical_accuracy: 0.0394 - factorized_top_k/top_50_categorical_accuracy: 0.1770 - factorized_top_k/top_100_categorical_accuracy: 0.3022 - loss: 66035.5178 - regularization_loss: 0.0000e+00 - total_loss: 66035.5178
Epoch 2/3
10/10 [==============================] - 24s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0032 - factorized_top_k/top_5_categorical_accuracy: 0.0259 - factorized_top_k/top_10_categorical_accuracy: 0.0529 - factorized_top_k/top_50_categorical_accuracy: 0.2121 - factorized_top_k/top_100_categorical_accuracy: 0.3474 - loss: 64954.1349 - regularization_loss: 0.0000e+00 - total_loss: 64954.1349
Epoch 3/3
10/10 [==============================] - 24s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0036 - factorized_top_k/top_5_categorical_accuracy: 0.0293

In [ ]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 10s 954ms/step - factorized_top_k/top_1_categorical_accuracy: 7.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0095 - factorized_top_k/top_10_categorical_accuracy: 0.0223 - factorized_top_k/top_50_categorical_accuracy: 0.1246 - factorized_top_k/top_100_categorical_accuracy: 0.2327 - loss: 31079.0615 - regularization_loss: 0.0000e+00 - total_loss: 31079.0615


{'factorized_top_k/top_100_categorical_accuracy': 0.23274999856948853,
 'factorized_top_k/top_10_categorical_accuracy': 0.022299999371170998,
 'factorized_top_k/top_1_categorical_accuracy': 0.000699999975040555,
 'factorized_top_k/top_50_categorical_accuracy': 0.12460000067949295,
 'factorized_top_k/top_5_categorical_accuracy': 0.009499999694526196,
 'loss': 28244.771484375,
 'regularization_loss': 0,
 'total_loss': 28244.771484375}

In [ ]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

In [ ]:
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

In [ ]:
# Get recommendations.
_, titles = index(tf.constant(["27"]))
print(f"Recommendations for user 27: {titles[0, :3]}")

Recommendations for user 27: [b'Bottle Rocket (1996)' b'Trigger Effect, The (1996)'
 b'Last Supper, The (1995)']
